In [2]:
import pandas as pd
import stats

data_dir = 'D:\FINKI\8_dps\Project\DATA\physionet_dbs\cudb\clean_segments_ft_250hz'

In [3]:
dataset = pd.read_csv(f'{data_dir}\\ft_segments_3s.csv')
dataset = dataset[dataset['episode'].isin(['N', 'VF'])]

# TO BE CONTINUED